### 3-3．모델작성
#### 데이터의 리드와 확인

In [1]:
# Pandas의 로드
import pandas as pd

# 파일 리드
bank_df_new = pd.read_csv('bank-prep.csv', sep=',')

# 선두에서 5행까지 표시
bank_df_new.head()

age  default  balance  housing  loan  day  duration  campaign  pdays  \
0   58        0     2143        1     0    5       261         1     -1   
1   36        0      265        1     1    5       348         1     -1   
2   25        0       -7        1     0    5       365         1     -1   
3   53        0       -3        0     0    5      1666         1     -1   
4   24        0     -103        1     1    5       145         1     -1   

   previous  ...  dec  feb  jan  jul  jun  mar  may  nov  oct  sep  
0         0  ...    0    0    0    0    0    0    1    0    0    0  
1         0  ...    0    0    0    0    0    0    1    0    0    0  
2         0  ...    0    0    0    0    0    0    1    0    0    0  
3         0  ...    0    0    0    0    0    0    1    0    0    0  
4         0  ...    0    0    0    0    0    0    1    0    0    0  

[5 rows x 32 columns]

#### 불균형데이터의 균형화

In [2]:
!pip show imbalanced-learn

Name: imbalanced-learn
Version: 0.4.3
Summary: Toolbox for imbalanced dataset in machine learning.
Home-page: https://github.com/scikit-learn-contrib/imbalanced-learn
Author: None
Author-email: None
License: MIT
Location: e:\anaconda3\envs\pbook_appx\lib\site-packages
Requires: scikit-learn, scipy, numpy
Required-by: 


In [3]:
!pip list

Package             Version   
------------------- ----------
absl-py             0.7.0     
astor               0.7.1     
backcall            0.1.0     
bleach              3.1.0     
certifi             2018.11.29
colorama            0.4.1     
cycler              0.10.0    
decorator           4.3.2     
entrypoints         0.3       
gast                0.2.2     
graphviz            0.10.1    
grpcio              1.16.1    
h5py                2.8.0     
imbalanced-learn    0.4.3     
ipykernel           5.1.0     
ipython             7.2.0     
ipython-genutils    0.2.0     
ipywidgets          7.4.2     
Janome              0.3.7     
jedi                0.13.2    
Jinja2              2.10      
jsonschema          2.6.0     
jupyter             1.0.0     
jupyter-client      5.2.4     
jupyter-console     6.0.0     
jupyter-core        4.4.0     
jupyterlab          0.35.3    
jupyterlab-server   0.2.0     
Keras               2.2.4     
Keras-Applications  1.0.6     
Keras-Pr

In [4]:
!pip install imbalanced-learn

In [5]:
# Numpy와 RandomUnderSampler의 로드
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

# 데이터 세트를 설명변수와 목적변수로 분할
X = np.array(bank_df_new.drop('y', axis=1))
Y = np.array(bank_df_new['y'])

# y가 1과 0인 데이터의 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

# y가 1인 데이터의 건수에 맞춰서 다운사이징
sampler = RandomUnderSampler(random_state=42)
X, Y = sampler.fit_resample(X, Y)

# y가 1과 0인 데이터 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

820 6113
820 820


#### 결정트리의 작성과 검증

In [6]:
# Scikit-learn의 로드
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.metrics import accuracy_score

# K-Fold 교차검증
kf = KFold(n_splits=10, shuffle=True)
# 모델의 정밀도를 보존할 준비
scores = []
# 데이터를 셔플하고, 훈련데이터와 테스트데이트로 분할
for train_id, test_id in kf.split(X):
    # 훈련 데이터를 사용하여 모델을 작성
    x = X[train_id]
    y = Y[train_id]
    clf = tree.DecisionTreeClassifier()
    clf.fit(x,y)
    # 테스트 데이터에 모델을 적용
    pred_y = clf.predict(X[test_id])
    # 모델의 정밀도를 계산해서 보존
    score = accuracy_score(Y[test_id], pred_y)
    scores.append(score)

# 모델의 평균정밀도, 표준편차를 확인
scores = np.array(scores)
print(scores.mean(), scores.std())

0.7640243902439025 0.03168972307362511


In [7]:
# Scikit-learn의 로드
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# 재현률, 적합률의 확인
#print(confusion_matrix(Y[test_id], pred_y))
print(recall_score(Y[test_id], pred_y))
print(precision_score(Y[test_id], pred_y))

0.7710843373493976
0.7710843373493976


#### 모델의 정밀도 향상

In [8]:
# 모델작성용 패러메터를 확인
print(clf)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [9]:
# Scikit-learn을 로드
from sklearn.model_selection import GridSearchCV

# 패러메터 범위설정
params = {
    'criterion': ['entropy'],    
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [10, 20, 30, 40, 50],
}

# 그리드 서치의 실행조건을 설정
clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params, 
                      cv=KFold(n_splits=10, shuffle=True), scoring='accuracy')

# 그리드 서치 실행
clf_gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['entropy'], 'max_depth': [2, 4, 6, 8, 10], 'min_samples_leaf': [10, 20, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [10]:
# 가장 높은 정밀도와 패터메터의 조합을 표시
print(clf_gs.best_score_)
print(clf_gs.best_params_)

0.801829268292683
{'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 20}


In [11]:
# 최적의 패터메터를 사용한 모델을 작성
clf_best = tree.DecisionTreeClassifier(
    criterion='entropy', max_depth=10, min_samples_leaf=20)
clf_best.fit(X, Y)

# 영향이 높은 변수를 확인
print(clf_best.feature_importances_)

[0.01496426 0.         0.04498507 0.09824143 0.01612625 0.0424749
 0.54403349 0.00916479 0.06363245 0.03981101 0.         0.00558156
 0.         0.         0.00303882 0.         0.         0.
 0.07026309 0.         0.01300379 0.         0.         0.
 0.00349388 0.         0.         0.0311852  0.         0.
 0.        ]
